In [ ]:
import os
import numpy as np
import librosa as lr
import tensorflow as tf

In [ ]:
genres = {0: "Hip-Pop", 
          1: "Pop", 
          2: "Folk", 
          3: "Experimental", 
          4: "Rock", 
          5: "International", 
          6: "Electronic", 
          7: "Instrumental"}

In [ ]:
input_data = []

for track in os.listdir("./data/input/"):
    if track[-4:] == ".mp3":
        input_track = [track[:-4]]
        
        try:
            y, sr = lr.load(f"./data/input/{track}", sr=None)
            
            features = []
            num_seg = len(y) // (5 * sr)
            
            for i in range(num_seg):
                y_idx = i * (5 * sr)
                y_seg = y[y_idx:y_idx + (5 * sr)]
                
                hl = int((sr / 44100) * 2048)
                
                feature = lr.feature.melspectrogram(y_seg, sr, n_mels=128, hop_length=hl)
                feature = np.array(feature).reshape(-1, 128, 108, 1)
                
                features.append(feature)

            input_track.append(features)
        except Exception as e:
            pass
        
        input_data.append(input_track)

In [ ]:
for track in input_data:
    print(f"Track Title: {track[0]:<31} Total Segments: {len(track[1])}")

In [ ]:
model = tf.keras.models.load_model("./models/final.model")

In [ ]:
for track in input_data:
    predicted_genres = {v: 0 for k, v in genres.items()}
    for feature in track[1]:
        prediction = model.predict(feature)
        predicted_genre = genres[np.argmax(prediction)]
        
        predicted_genres[predicted_genre] = predicted_genres[predicted_genre] + 1

    top_genre = [genres[0], 0]
    for k, v in predicted_genres.items():
        if v > top_genre[1] and k != "Experimental":
            top_genre = [k, v]
        
    track.append(top_genre[0])
    track.append(predicted_genres)

In [ ]:
for track in input_data:
    print(f"Track Title: {track[0]:<31} Top Genre: {track[2]}")

In [ ]:
for track in input_data:
    print(f"\nTrack Title: {track[0]:<31}")
    
    for k, v in track[3].items():
        print(f" - {k:<13}: {v:}")